# 針對Oxford Robotcar Dataset 做前處理

1. 去拜耳化
2. 調整尺寸
3. 生成分割影像(需要利用NVIDIA 的模型生成)
4. 每隔幾張影像進行刪除，得到差異較大的影像
5. 調整數字排序
6. 生成訓練集和測試集的文件

### 載入debayer 相關套件

In [5]:
from PIL import Image
from colour_demosaicing import demosaicing_CFA_Bayer_bilinear as demosaic
import numpy as np

import re
import os
import numpy as np
import scipy.interpolate as interp
from scipy.ndimage import map_coordinates


class CameraModel:


    def __init__(self, models_dir, images_dir):

        self.camera = None
        self.camera_sensor = None
        self.focal_length = None
        self.principal_point = None
        self.G_camera_image = None
        self.bilinear_lut = None

        self.__load_intrinsics(models_dir, images_dir)
        self.__load_lut(models_dir, images_dir)

    def project(self, xyz, image_size):
      
        if xyz.shape[0] == 3:
            xyz = np.stack((xyz, np.ones((1, xyz.shape[1]))))
        xyzw = np.linalg.solve(self.G_camera_image, xyz)

        # Find which points lie in front of the camera
        in_front = [i for i in range(0, xyzw.shape[1]) if xyzw[2, i] >= 0]
        xyzw = xyzw[:, in_front]

        uv = np.vstack((self.focal_length[0] * xyzw[0, :] / xyzw[2, :] + self.principal_point[0],
                        self.focal_length[1] * xyzw[1, :] / xyzw[2, :] + self.principal_point[1]))

        in_img = [i for i in range(0, uv.shape[1])
                  if 0.5 <= uv[0, i] <= image_size[1] and 0.5 <= uv[1, i] <= image_size[0]]

        return uv[:, in_img], np.ravel(xyzw[2, in_img])

    def undistort(self, image):
      
        if image.shape[0] * image.shape[1] != self.bilinear_lut.shape[0]:
            raise ValueError('Incorrect image size for camera model')

        lut = self.bilinear_lut[:, 1::-1].T.reshape((2, image.shape[0], image.shape[1]))

        if len(image.shape) == 1:
            raise ValueError('Undistortion function only works with multi-channel images')

        undistorted = np.rollaxis(np.array([map_coordinates(image[:, :, channel], lut, order=1)
                                for channel in range(0, image.shape[2])]), 0, 3)

        return undistorted.astype(image.dtype)

    def __get_model_name(self, images_dir):
        self.camera = re.search('(stereo|mono_(left|right|rear))', images_dir).group(0)
        if self.camera == 'stereo':
            self.camera_sensor = re.search('(left|centre|right)', images_dir).group(0)
            if self.camera_sensor == 'left':
                return 'stereo_wide_left'
            elif self.camera_sensor == 'right':
                return 'stereo_wide_right'
            elif self.camera_sensor == 'centre':
                return 'stereo_narrow_left'
            else:
                raise RuntimeError('Unknown camera model for given directory: ' + images_dir)
        else:
            return self.camera

    def __load_intrinsics(self, models_dir, images_dir):
        model_name = self.__get_model_name(images_dir)
        intrinsics_path = os.path.join(models_dir, model_name + '.txt')

        with open(intrinsics_path) as intrinsics_file:
            vals = [float(x) for x in next(intrinsics_file).split()]
            self.focal_length = (vals[0], vals[1])
            self.principal_point = (vals[2], vals[3])

            G_camera_image = []
            for line in intrinsics_file:
                G_camera_image.append([float(x) for x in line.split()])
            self.G_camera_image = np.array(G_camera_image)

    def __load_lut(self, models_dir, images_dir):
        model_name = self.__get_model_name(images_dir)
        lut_path = os.path.join(models_dir, model_name + '_distortion_lut.bin')

        lut = np.fromfile(lut_path, np.double)
        lut = lut.reshape([2, lut.size // 2])
        self.bilinear_lut = lut.transpose()


BAYER_STEREO = 'gbrg'
BAYER_MONO = 'rggb'


def load_image(image_path, model=None):

    if model:
        camera = model.camera
    else:
        camera = re.search('(stereo|mono_(left|right|rear))', image_path).group(0)
    if camera == 'stereo':
        pattern = BAYER_STEREO
    else:
        pattern = BAYER_MONO

    img = Image.open(image_path)
    img = demosaic(img, pattern)
    if model:
        img = model.undistort(img)

    return np.array(img).astype(np.uint8)

### 執行 debayer 

In [75]:

import argparse
import os
import re
import matplotlib.pyplot as plt
from PIL import Image
from datetime import datetime as dt
import cv2 
import pathlib


def debayer(dir):

    models_dir = None

    camera = re.search('(stereo|mono_(left|right|rear))', dir).group(0)

    timestamps_path = os.path.join(os.path.join(dir, os.pardir, camera + '.timestamps'))

    if not os.path.isfile(timestamps_path):
        timestamps_path = os.path.join(dir, os.pardir, os.pardir, camera + '.timestamps')
        if not os.path.isfile(timestamps_path):
            raise IOError("Could not find timestamps file")
    
    model = None
    if models_dir:
        model = CameraModel(models_dir, dir)
        
    current_chunk = 0
    timestamps_file = open(timestamps_path)

    count = 0
    for file in os.listdir(dir):
        count = count + 1

    file_count = 1

    for line in timestamps_file:
        
        tokens = line.split()
        datetime = dt.utcfromtimestamp(int(tokens[0])/1000000)
        chunk = int(tokens[1])

        filename = os.path.join(dir, tokens[0] + '.png')

        if not os.path.isfile(filename):
            if chunk != current_chunk:
                print("Chunk " + str(chunk) + " not found")
                current_chunk = chunk
            continue

        current_chunk = chunk
        img = load_image(filename, model)
        img = Image.fromarray(img)
        
        path_dir = os.path.abspath(os.path.dirname(os.path.dirname(dir)))
       
        debayer_path = path_dir + '/debayer'
        if not os.path.isdir(debayer_path):
            print('建立 debayer 資料夾...')
            os.makedirs(debayer_path)
        
        img.save(debayer_path + '/' + tokens[0] + '.png')
        print('目前進行 debayer 進度為： [',file_count,'/',count,']')
        file_count += 1
    print('已處理完畢 ',dir)

if __name__ == '__main__':
    file_path = '/home/joy/masterthesis/dataset/robotcar/offical/2014-12-10/2014-12-10-18-10-50_stereo_left_07/2014-12-10-18-10-50/stereo/left/'
    debayer(file_path)

Chunk 1 not found
Chunk 2 not found
Chunk 3 not found
Chunk 4 not found
Chunk 5 not found
Chunk 6 not found
建立 debayer 資料夾...
目前進行 debayer 進度為： [ 1 / 2314 ]
目前進行 debayer 進度為： [ 2 / 2314 ]
目前進行 debayer 進度為： [ 3 / 2314 ]
目前進行 debayer 進度為： [ 4 / 2314 ]
目前進行 debayer 進度為： [ 5 / 2314 ]
目前進行 debayer 進度為： [ 6 / 2314 ]
目前進行 debayer 進度為： [ 7 / 2314 ]
目前進行 debayer 進度為： [ 8 / 2314 ]
目前進行 debayer 進度為： [ 9 / 2314 ]
目前進行 debayer 進度為： [ 10 / 2314 ]
目前進行 debayer 進度為： [ 11 / 2314 ]
目前進行 debayer 進度為： [ 12 / 2314 ]
目前進行 debayer 進度為： [ 13 / 2314 ]
目前進行 debayer 進度為： [ 14 / 2314 ]
目前進行 debayer 進度為： [ 15 / 2314 ]
目前進行 debayer 進度為： [ 16 / 2314 ]
目前進行 debayer 進度為： [ 17 / 2314 ]
目前進行 debayer 進度為： [ 18 / 2314 ]
目前進行 debayer 進度為： [ 19 / 2314 ]
目前進行 debayer 進度為： [ 20 / 2314 ]
目前進行 debayer 進度為： [ 21 / 2314 ]
目前進行 debayer 進度為： [ 22 / 2314 ]
目前進行 debayer 進度為： [ 23 / 2314 ]
目前進行 debayer 進度為： [ 24 / 2314 ]
目前進行 debayer 進度為： [ 25 / 2314 ]
目前進行 debayer 進度為： [ 26 / 2314 ]
目前進行 debayer 進度為： [ 27 / 2314 ]
目前進行 debayer 進度為： [

# 調整影像尺寸

In [81]:

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
from IPython.display import clear_output

def crop_img(img):
    # 1152,672
    # left, right
    x_l, x_r = 64, 1216

    # up, down
    y_u, y_d = 0, 672 

    # crop image
    crop_img = img[y_u:y_d, x_l:x_r]  # notice: first y, then x

    return crop_img

def show_img(img):
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(image_rgb)
    plt.show()

def img_processing(img):
    img = crop_img(img)

    return img


def read_path(file_pathname):
    # 遍歷該目錄下的所有圖片檔案
    count = 0
    for file in os.listdir(file_pathname):
        count = count + 1

    file_count = 1

    for filename in os.listdir(file_pathname):

        dir = os.path.abspath(os.path.dirname(os.path.dirname(file_pathname)))
        resize_path = dir + '/resize'
        
        if not os.path.isdir(resize_path):
            print('建立 resize 資料夾...')
            os.makedirs(resize_path)

        origin_img = cv2.imread(file_pathname + '/' + filename)

        result_img = img_processing(origin_img)


        cv2.imwrite(resize_path + '/' + filename, result_img)      
        print('目前進行 resize 進度為[',file_count,'/',count,']')
        file_count += 1

    print('已處理完畢 ',file_pathname)

if __name__ == '__main__':
    file_path = "/home/joy/masterthesis/dataset/robotcar/offical/2014-12-10/2014-12-10-18-10-50_stereo_left_07/2014-12-10-18-10-50/stereo/debayer/"
    read_path(file_path)


建立 resize 資料夾...
目前進行 resize 進度為[ 1 / 2314 ]
目前進行 resize 進度為[ 2 / 2314 ]
目前進行 resize 進度為[ 3 / 2314 ]
目前進行 resize 進度為[ 4 / 2314 ]
目前進行 resize 進度為[ 5 / 2314 ]
目前進行 resize 進度為[ 6 / 2314 ]
目前進行 resize 進度為[ 7 / 2314 ]
目前進行 resize 進度為[ 8 / 2314 ]
目前進行 resize 進度為[ 9 / 2314 ]
目前進行 resize 進度為[ 10 / 2314 ]
目前進行 resize 進度為[ 11 / 2314 ]
目前進行 resize 進度為[ 12 / 2314 ]
目前進行 resize 進度為[ 13 / 2314 ]
目前進行 resize 進度為[ 14 / 2314 ]
目前進行 resize 進度為[ 15 / 2314 ]
目前進行 resize 進度為[ 16 / 2314 ]
目前進行 resize 進度為[ 17 / 2314 ]
目前進行 resize 進度為[ 18 / 2314 ]
目前進行 resize 進度為[ 19 / 2314 ]
目前進行 resize 進度為[ 20 / 2314 ]
目前進行 resize 進度為[ 21 / 2314 ]
目前進行 resize 進度為[ 22 / 2314 ]
目前進行 resize 進度為[ 23 / 2314 ]
目前進行 resize 進度為[ 24 / 2314 ]
目前進行 resize 進度為[ 25 / 2314 ]
目前進行 resize 進度為[ 26 / 2314 ]
目前進行 resize 進度為[ 27 / 2314 ]
目前進行 resize 進度為[ 28 / 2314 ]
目前進行 resize 進度為[ 29 / 2314 ]
目前進行 resize 進度為[ 30 / 2314 ]
目前進行 resize 進度為[ 31 / 2314 ]
目前進行 resize 進度為[ 32 / 2314 ]
目前進行 resize 進度為[ 33 / 2314 ]
目前進行 resize 進度為[ 34 / 2314 ]
目前進行 r

# 增加影像之間的差異，移除部份影像

In [15]:
from cgi import print_environ
import os

def del_img(path):
    for dirPath, dirNames, fileNames in os.walk(path):
        dirNames.sort()
        fileNames.sort()
        img = []
    
        for file in fileNames:
            img.append(file)

        print('目前影像總數 : ',len(img))
        print('預計移除的影像數量 : ',len(set(img).difference(set(img[::3]))))
        print('預計留下的影像數量 : ',len(img[::3]))
   
        save = set(img).difference(set(img[::3]))
        for item in save :
            os.remove(path +'//'+ item)

        print('已移除完畢,請勿再次執行！')


if __name__ == '__main__':
    # file_path = "/home/joy/masterthesis/Joy/Robotcar_full/2014-12-16-18-44-24_stereo_left_06/2014-12-16-18-44-24/stereo/left"
    file_path = '/home/joy/masterthesis/Joy/Robotcar_full/segmentionation/2014-12-16-18-44-24_stereo_left_06/2014-12-16-18-44-24/stereo/left'
    del_img(file_path)


目前影像總數 :  10898
預計移除的影像數量 :  7265
預計留下的影像數量 :  3633
已移除完畢,請勿再次執行！


# 對影像進行排序及編號

In [25]:

import os

def convert(dir):
    file_list = os.listdir(dir)
    count = 0
    # 影像開始的編號
    n = 18880
    file_list.sort()

    # print(file_list)
    for filename in file_list:
        oldname = dir + filename

        # 設置新文件名
        s = str(n + 1)

        # 數字名稱的總位數
        st = s.zfill(6)
        newname = dir + st + '.png'

        # 對文件修改名字
        os.rename(oldname,newname)
        print(filename,'====》',st + '.png')

        n += 1
        count += 1
    print('已將 ',count,' 張影像修改編號')

if __name__ == '__main__':

    # segmentation 影像
    segmentation_dir = '/home/joy/masterthesis/Joy/Robotcar_full/segmentionation/2014-12-16-18-44-24_stereo_left_06/2014-12-16-18-44-24/stereo/left/'
    convert(segmentation_dir)

    # RGB 影像
    RGB_dir = '/home/joy/masterthesis/Joy/Robotcar_full/2014-12-16-18-44-24_stereo_left_06/2014-12-16-18-44-24/stereo/left/'
    convert(RGB_dir)

1418236968556930.png ====》 018881.png
1418236968744405.png ====》 018882.png
1418236968931879.png ====》 018883.png
1418236969119354.png ====》 018884.png
1418236969306829.png ====》 018885.png
1418236969494303.png ====》 018886.png
1418236969681777.png ====》 018887.png
1418236969869253.png ====》 018888.png
1418236970056727.png ====》 018889.png
1418236970244199.png ====》 018890.png
1418236970431674.png ====》 018891.png
1418236970619150.png ====》 018892.png
1418236970806624.png ====》 018893.png
1418236970994099.png ====》 018894.png
1418236971181574.png ====》 018895.png
1418236971369049.png ====》 018896.png
1418236971556523.png ====》 018897.png
1418236971743997.png ====》 018898.png
1418236971931472.png ====》 018899.png
1418236972118947.png ====》 018900.png
1418236972306420.png ====》 018901.png
1418236972493896.png ====》 018902.png
1418236972681382.png ====》 018903.png
1418236972868844.png ====》 018904.png
1418236973056319.png ====》 018905.png
1418236973243794.png ====》 018906.png
141823697343

# 生成訓練集與測試集的文件

In [31]:
import os
import numpy as np

def train_test_parting(root, model_name):

    # 建構所有檔案名稱的列表
    filename = []
    # 找到底下的資料夾 /home/joy/masterthesis/dataset/robotcar/offical/2014-12-10
    dirs = os.listdir(root)
    # 2014-12-10-18-10-50_stereo_left_01  2014-12-10-18-10-50_stereo_left_02...

    for dir in dirs:
        dir_path = root + '/' + dir
        if not 'segmentation' in dir_path:
            print('第一層目錄: ',dir_path)

            if os.path.isdir(dir_path):
                s_names = os.listdir(dir_path)[0]
                s_path = dir_path + '/' + s_names
                print('第二層目錄: ',s_path)

            if os.path.isdir(s_path):
                t_names = os.listdir(s_path)[0]
                t_path = s_path + '/' + t_names   
                print('第三層目錄: ',t_path)

            if os.path.isdir(t_path):
                f_names = os.listdir(t_path)[0]
                f_path = t_path + '/' + f_names   
                print('第四層目錄: ',f_path)

                names = os.listdir(f_path)
                # print('names ',names)

            key = '2014-12-16-18-44-24_stereo_left_0'
            # 找到隸屬於哪個資料夾底下
            if key in dir:
                num = dir.replace(key, '')      

            # 所有影像的名稱寫到文件裡
            for n in names:
                filename.append(''.join( x for x in n if x not in '.png') + '\t' + num)

    # 亂數處理
    np.random.shuffle(filename)

    # 劃分訓練集、測試集
    train = filename[:int(len(filename)*0.8)]
    print('訓練集共有: ',int(len(train)),'資料')
    test = filename[int(len(filename)*0.8):]
    print('測試集共有: ',int(len(test)),'資料')


    save_path = './splits/'+ model_name
    
    if not os.path.isdir(save_path):
        print('建立 splits 資料夾...')
        os.makedirs(save_path)

    # 分別寫入train.txt, test.txt
    with open(save_path + '/train_files.txt', 'w') as f1, open(save_path + '/test_files.txt', 'w') as f2:

        for i in train:
            f1.write(i + '\n')
        for k in test:
            f2.write(k + '\n')

    print('成功！')

if __name__ == '__main__':

    file_path = '/home/joy/masterthesis/Joy/Robotcar_full'
    model_name = 'robotcar_full'
    train_test_parting(file_path, model_name)

第一層目錄:  /home/joy/masterthesis/Joy/Robotcar_full/2014-12-16-18-44-24_stereo_left_01
第二層目錄:  /home/joy/masterthesis/Joy/Robotcar_full/2014-12-16-18-44-24_stereo_left_01/2014-12-16-18-44-24
第三層目錄:  /home/joy/masterthesis/Joy/Robotcar_full/2014-12-16-18-44-24_stereo_left_01/2014-12-16-18-44-24/stereo
第四層目錄:  /home/joy/masterthesis/Joy/Robotcar_full/2014-12-16-18-44-24_stereo_left_01/2014-12-16-18-44-24/stereo/left
names  ['001870.png', '001066.png', '002511.png', '000031.png', '000531.png', '000292.png', '000346.png', '001795.png', '001843.png', '001983.png', '000913.png', '001424.png', '001672.png', '001788.png', '001119.png', '001436.png', '000094.png', '001809.png', '002011.png', '002598.png', '000062.png', '001705.png', '001016.png', '000850.png', '001038.png', '002675.png', '001408.png', '001568.png', '002604.png', '001391.png', '000583.png', '000517.png', '000983.png', '000089.png', '002164.png', '001319.png', '002521.png', '001935.png', '001817.png', '000508.png', '002697.png', '00